In [1]:
import signal
import threading
from collections import defaultdict
from typing import Dict

from ipyleaflet import Circle, LayerGroup, Map
from palettable.colorbrewer.qualitative import Set1_9, Set2_6
from visionlib.pipeline.consumer import RedisConsumer
from visionapi_yq.messages_pb2 import SaeMessage

STREAMS = [
    'trackletmerger:stream1',
    'trackletmerger:stream2'
]
REDIS_PORT = 6379

# Setup graceful exit on SIGINT
stop_event = threading.Event()

def sig_handler(signum, _):
    signame = signal.Signals(signum).name
    print(f'Caught signal {signame} ({signum}). Exiting...')
    stop_event.set()

signal.signal(signal.SIGTERM, sig_handler)
signal.signal(signal.SIGINT, sig_handler)

CLS_CMAP = Set1_9.hex_colors
STREAM_CMAP = Set1_9.hex_colors

def get_stream_color(stream_key):
    try:
        idx = STREAMS.index(stream_key)
        return STREAM_CMAP[idx]
    except ValueError:
        return (255,255,255)

# Map setup
INIT_CENTER = (39.975605748738964, -86.12710274800055)
DETECTED_CENTER = None

m = Map(center=INIT_CENTER, zoom=19)
marker_layer = LayerGroup()
m.add(marker_layer)
m.layout.height = '800px'
display(m)

displayed_objects: Dict[str, Dict[str, Circle]] = defaultdict(lambda: defaultdict(dict))

# Start listening to SAE messages
consume = RedisConsumer('localhost', REDIS_PORT, STREAMS, block=200)

with consume:
    for stream_key, proto_data in consume():

        if stop_event.is_set():
            break

        if stream_key is None:
            continue
        
        stream_id = stream_key.split(':')[1]
   
        sae_msg = SaeMessage()
        sae_msg.ParseFromString(proto_data)     

        active_track_ids = set(track_id for track_id in sae_msg.trajectory.cameras[stream_id].tracklets)
        removed_objects = [id for id in displayed_objects[stream_id].keys() if id not in active_track_ids]
        for id in removed_objects:
            removed = displayed_objects[stream_id].pop(id, None)
            if removed is not None:
                marker_layer.remove(removed)
        # Remove objects from cache and map that were rendered before but do not exist anymore
        # removed_objects = list(filter(lambda id: id not in map(lambda t: t.track_id, sae_msg.trajectory.cameras[stream_key].tracklets), displayed_objects[stream_key].keys()))
        # for id in removed_objects:
        #     removed = displayed_objects[stream_key].pop(id, None)
        #     if removed is not None:
        #         marker_layer.remove(removed)

        # Add new objects / Update existing objects locations
        print(f'In stream: {stream_key}, the total number of detections is {len(sae_msg.trajectory.cameras[stream_id].tracklets)}')
        for idx, track_id in enumerate(sae_msg.trajectory.cameras[stream_id].tracklets):
            detection = sae_msg.trajectory.cameras[stream_id].tracklets[track_id].detections_info[-1] # the latest detection of each tracklet

            if not detection.HasField('geo_coordinate'):
                continue
            
            # Update map center with the first detected object (should only happen once)
            if DETECTED_CENTER is None and len(sae_msg.detections) > 0:
                DETECTED_CENTER = (detection.geo_coordinate.latitude, detection.geo_coordinate.longitude)
                m.center = DETECTED_CENTER
                m.zoom = 19

            lat, lon = detection.geo_coordinate.latitude, detection.geo_coordinate.longitude
            if track_id in displayed_objects[stream_id]:
                displayed_objects[stream_id][track_id].location = (lat, lon)
            else:
                marker = Circle(location=[lat, lon], radius=1, color=get_stream_color(stream_key), fill_color=CLS_CMAP[detection.class_id], fill_opacity=0.8)
                displayed_objects[stream_id][track_id] = marker
                marker_layer.add(marker)

Map(center=[39.975605748738964, -86.12710274800055], controls=(ZoomControl(options=['position', 'zoom_in_text'…

ConnectionError: Error 111 connecting to localhost:6379. Connection refused.

# Input serves as detection


In [1]:
import signal
import random
import threading
from collections import defaultdict
from typing import Dict

from ipywidgets import HTML
from ipyleaflet import Circle, LayerGroup, Map, Popup, Marker, Icon, Polygon
from palettable.colorbrewer.qualitative import Set1_9, Set2_6
from visionapi_yq.messages_pb2 import SaeMessage
from visionlib.pipeline.consumer import RedisConsumer


import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

STREAMS = [
    'trackletmerger:stream1',
    'trackletmerger:stream2'
]
REDIS_PORT = 6379

# Setup graceful exit on SIGINT
stop_event = threading.Event()

def draw_polygon_from_file(map_obj, filename):
    # 1. Read the polygon points
    polygon_coords = []
    with open(filename, "r") as f:
        for line in f:
            lat_str, lon_str = line.strip().split()
            # Convert strings to float and store as [lat, lon]
            polygon_coords.append([float(lat_str), float(lon_str)])

    rand_color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    # 2. Create a Polygon layer.
    #    Note that `locations` for ipyleaflet.Polygon expects a list of "rings" 
    #    and each ring is a list of [lat, lon] pairs.
    #    For a simple polygon with no holes, we have just one ring (outer boundary).
    polygon_layer = Polygon(
        locations=[polygon_coords],
        color=rand_color,       # outline color
        fill_color=rand_color,    # fill color
        fill_opacity=0.2,    # adjust fill transparency as you like
        name="My Polygon"    # optional name for legend or debugging
    )

    # 3. Add the polygon to the map
    map_obj.add_layer(polygon_layer)

def sig_handler(signum, _):
    signame = signal.Signals(signum).name
    print(f'Caught signal {signame} ({signum}). Exiting...')
    stop_event.set()

signal.signal(signal.SIGTERM, sig_handler)
signal.signal(signal.SIGINT, sig_handler)

CLS_CMAP = Set1_9.hex_colors
STREAM_CMAP = Set1_9.hex_colors

def get_stream_color(stream_key):
    try:
        idx = STREAMS.index(stream_key)
        return STREAM_CMAP[idx]
    except ValueError:
        return (255,255,255)

# Map setup
INIT_CENTER = (39.97559002347724, -86.1270794622172)
DETECTED_CENTER = None

m = Map(center=INIT_CENTER, zoom=18)
marker_layer = LayerGroup()
m.add(marker_layer)
m.layout.height = '1300px'

draw_polygon_from_file(m, "polygon_points_NB.txt")
draw_polygon_from_file(m, "polygon_points_SB.txt")
display(m)

displayed_objects: Dict[str, Dict[str, Circle]] = defaultdict(lambda: defaultdict(dict))

stream_id_list = ['stream1','stream2']

track_id_database = {idx: [] for idx in stream_id_list}

# Start listening to SAE messages
consume = RedisConsumer('localhost', REDIS_PORT, STREAMS, block=200)

with consume:
    for stream_key, proto_data in consume():
        if stop_event.is_set():
            break

        if stream_key is None:
            continue
        
        stream_id = stream_key.split(':')[1]
        sae_msg = SaeMessage()
        sae_msg.ParseFromString(proto_data)

        # Remove objects from cache and map that were rendered before but do not exist anymore
        removed_objects = list(
            filter(
                lambda id: id not in map(lambda d: d.object_id, sae_msg.detections),
                displayed_objects[stream_id].keys(),
            )
        )

        for id in removed_objects:
            removed = displayed_objects[stream_id].pop(id, None)
            if removed is not None:
                marker_layer.remove(removed['circle'])  # Remove the Circle
                m.remove_layer(removed['label'])       # Remove the Label (Marker)

        # Add new objects / Update existing objects locations
        # print(f'In stream: {stream_id}, the total number of detections is {len(sae_msg.detections)}')
        for detection in sae_msg.detections:
            if not detection.HasField('geo_coordinate'):
                continue

            lat, lon = detection.geo_coordinate.latitude, detection.geo_coordinate.longitude

            # Check if object is been matched
            flag = False  # Reset flag for each detection
            for str_id in stream_id_list:
                if str_id == stream_id:
                    continue
                else:
                    if detection.object_id in track_id_database[str_id]:
                        flag = True
                        print('flag=True')
                        break

            highlight_color = '#00FF00'   # Green for matched objects
            default_color = get_stream_color(stream_key)  # Default stream color

            if detection.object_id in displayed_objects[stream_id]:
                displayed_objects[stream_id][detection.object_id]['circle'].location = (lat, lon)
                displayed_objects[stream_id][detection.object_id]['label'].location = (lat, lon)
                displayed_objects[stream_id][detection.object_id]['circle'].color = highlight_color if flag else default_color
                displayed_objects[stream_id][detection.object_id]['circle'].fill_color = highlight_color if flag else CLS_CMAP[detection.class_id]
            else:
                # First time to be detected
                track_id_database[stream_id].append(detection.object_id)
                circle = Circle(location=[lat, lon], radius=1, color=get_stream_color(stream_key), fill_color=CLS_CMAP[detection.class_id], fill_opacity=0.8)
                icon = Icon(icon_url='', icon_size=(0, 0))
                label = Marker(
                    location=[lat, lon],
                    icon=icon,
                    draggable=False,
                    title=f"ID: {detection.object_id}"  # Text to show as label
                )
                displayed_objects[stream_id][detection.object_id] = {'circle': circle, 'label': label}
                
                # Add the circle and label to the map
                marker_layer.add(circle)
                m.add_layer(label)

Map(center=[39.97559002347724, -86.1270794622172], controls=(ZoomControl(options=['position', 'zoom_in_text', …

flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True
flag=True


ConnectionError: Connection closed by server.

In [9]:
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

# Configure Firefox options (headless mode)
options = Options()
options.headless = True

# Create a Firefox webdriver instance.
# If you updated geckodriver and it's in /usr/local/bin, you can specify the executable_path if needed.
driver = webdriver.Firefox(options=options)

# Use your notebook URL (include token if required)
notebook_url = "http://localhost:8888/notebooks/yl4300/project/Multi-Camera-Vision-Pipeline-YQ/tools/geo-mapping/geo-mapper-visualizer.ipynb?token=85280b5b8d4e7dbfe8118b67048f361f2f06491faaecb339"
driver.get(notebook_url)

# Wait for the notebook and map widget to load (adjust the sleep time if necessary)
time.sleep(10)

# Capture full-page screenshots
num_frames = 10  # Number of frames to capture
for i in range(num_frames):
    time.sleep(2)  # Wait between frames to capture dynamic updates
    screenshot_filename = f"frame_{i:02d}.png"
    driver.save_screenshot(screenshot_filename)
    print(f"Saved {screenshot_filename}")

driver.quit()


WebDriverException: Message: Process unexpectedly closed with status 1


# Geo-Mapper tester

In [5]:
import signal
import threading
from collections import defaultdict
from typing import Dict

from ipywidgets import HTML
from ipyleaflet import Circle, LayerGroup, Map, Popup, Marker, Icon
from palettable.colorbrewer.qualitative import Set1_9, Set2_6
from visionapi_yq.messages_pb2 import SaeMessage
from visionlib.pipeline.consumer import RedisConsumer

STREAMS = [
    'trackletmerger:stream1',
    'trackletmerger:stream2'
]
REDIS_PORT = 6379

# Setup graceful exit on SIGINT
stop_event = threading.Event()

def sig_handler(signum, _):
    signame = signal.Signals(signum).name
    print(f'Caught signal {signame} ({signum}). Exiting...')
    stop_event.set()

signal.signal(signal.SIGTERM, sig_handler)
signal.signal(signal.SIGINT, sig_handler)

CLS_CMAP = Set1_9.hex_colors
STREAM_CMAP = Set1_9.hex_colors

def get_stream_color(stream_key):
    try:
        idx = STREAMS.index(stream_key)
        return STREAM_CMAP[idx]
    except ValueError:
        return (255,255,255)

# Map setup
INIT_CENTER = (39.975605748738964, -86.12710274800055)
DETECTED_CENTER = None

m = Map(center=INIT_CENTER, zoom=24)
marker_layer = LayerGroup()
m.add(marker_layer)
m.layout.height = '800px'
display(m)

displayed_objects: Dict[str, Dict[str, Circle]] = defaultdict(lambda: defaultdict(dict))


# Start listening to SAE messages
consume = RedisConsumer('localhost', REDIS_PORT, STREAMS, block=200)

with consume:
    for stream_key, proto_data in consume():
        

        if stop_event.is_set():
            break

        if stream_key is None:
            continue
        
        stream_id = stream_key.split(':')[1]
        sae_msg = SaeMessage()
        sae_msg.ParseFromString(proto_data)


        # Remove objects from cache and map that were rendered before but do not exist anymore
        removed_objects = list(
            filter(
                lambda id: id not in map(lambda d: d.object_id, sae_msg.detections),
                displayed_objects[stream_id].keys(),
            )
        )

        for id in removed_objects:
            removed = displayed_objects[stream_id].pop(id, None)
            if removed is not None:
                marker_layer.remove(removed['circle'])  # Remove the Circle
                # m.remove_layer(removed['label'])       # Remove the Label (Marker)

        # Add new objects / Update existing objects locations
        # print(f'In stream: {stream_id}, the total number of detections is {len(sae_msg.detections)}')
        for detection in sae_msg.detections:
            if not detection.HasField('geo_coordinate'):
                continue
            
            # Update map center with the first detected object (should only happen once)
            if DETECTED_CENTER is None and len(sae_msg.detections) > 0:
                DETECTED_CENTER = (detection.geo_coordinate.latitude, detection.geo_coordinate.longitude)
                m.center = DETECTED_CENTER
                m.zoom = 19

            lat, lon = detection.geo_coordinate.latitude, detection.geo_coordinate.longitude

            # Check if object is been matched
            default_color = get_stream_color(stream_key)  # Default stream color


            if detection.object_id in displayed_objects[stream_id]:
                displayed_objects[stream_id][detection.object_id]['circle'].location = (lat, lon)
                # displayed_objects[stream_id][detection.object_id]['label'].location = (lat, lon)
            else:
                circle = Circle(location=[lat, lon], radius=1, color=get_stream_color(stream_key), fill_color=CLS_CMAP[detection.class_id], fill_opacity=0.8)
                icon = Icon(icon_url='', icon_size=(0, 0))
                # label = Marker(
                #     location=[lat, lon],
                #     icon=icon,
                #     draggable=False,
                #     title=f"ID: {detection.object_id}"  # Text to show as label
                # )
                displayed_objects[stream_id][detection.object_id] = {'circle': circle}
                # displayed_objects[stream_id][detection.object_id] = {'circle': circle, 'label': label}
                
                # Add the circle and label to the map
                marker_layer.add(circle)
                # m.add_layer(label)

Map(center=[39.975605748738964, -86.12710274800055], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Caught signal SIGINT (2). Exiting...


In [6]:
import signal
import threading
from collections import defaultdict
from typing import Dict

from ipyleaflet import Circle, LayerGroup, Map
from palettable.colorbrewer.qualitative import Set1_9
from visionlib.pipeline.consumer import RedisConsumer
from visionapi_yq.messages_pb2 import SaeMessage

STREAMS = [
    'trackletmerger:stream1',
    'trackletmerger:stream2'
]
REDIS_PORT = 6379

# Setup graceful exit on SIGINT
stop_event = threading.Event()

def sig_handler(signum, _):
    signame = signal.Signals(signum).name
    print(f'Caught signal {signame} ({signum}). Exiting...')
    stop_event.set()

signal.signal(signal.SIGTERM, sig_handler)
signal.signal(signal.SIGINT, sig_handler)

CLS_CMAP = Set1_9.hex_colors
STREAM_CMAP = Set1_9.hex_colors

def get_stream_color(stream_key):
    try:
        idx = STREAMS.index(stream_key)
        return STREAM_CMAP[idx]
    except ValueError:
        return (255, 255, 255)

# Map setup
INIT_CENTER = (39.975605748738964, -86.12710274800055)
DETECTED_CENTER = None

m = Map(center=INIT_CENTER, zoom=19)
marker_layer = LayerGroup()
m.add(marker_layer)
m.layout.height = '800px'
display(m)

displayed_objects: Dict[str, Dict[str, Circle]] = defaultdict(dict)

# Start listening to SAE messages
consume = RedisConsumer('localhost', REDIS_PORT, STREAMS, block=200)

with consume:
    for stream_key, proto_data in consume():
        if stop_event.is_set():
            break

        if stream_key is None:
            continue

        stream_id = stream_key.split(':')[1]

        sae_msg = SaeMessage()
        sae_msg.ParseFromString(proto_data)

        # Gather active track IDs
        active_track_ids = set(
            track_id for track_id in sae_msg.trajectory.cameras[stream_id].tracklets
        )

        # Remove objects no longer in the stream
        removed_objects = [
            track_id for track_id in displayed_objects[stream_id].keys()
            if track_id not in active_track_ids
        ]
        for track_id in removed_objects:
            marker = displayed_objects[stream_id].pop(track_id, None)
            if marker:
                marker_layer.remove(marker)

        # Update or add new objects
        print(f"In stream: {stream_key}, the total number of detections is {len(active_track_ids)}")
        for idx, track_id in enumerate(sae_msg.trajectory.cameras[stream_id].tracklets):
            latest_detection = sae_msg.trajectory.cameras[stream_id].tracklets[track_id].detections_info[-1] # the latest detection of each tracklet

            if not latest_detection.HasField('geo_coordinate'):
                continue

            lat, lon = latest_detection.geo_coordinate.latitude, latest_detection.geo_coordinate.longitude

            # Update map center with the first detected object (once only)
            if DETECTED_CENTER is None:
                DETECTED_CENTER = (lat, lon)
                m.center = DETECTED_CENTER
                m.zoom = 19

            if track_id in displayed_objects[stream_id]:
                # Update existing marker's location
                displayed_objects[stream_id][track_id].location = (lat, lon)
            else:
                # Create a new marker for the track ID
                marker = Circle(
                    location=[lat, lon],
                    radius=1,
                    color=get_stream_color(stream_key),
                    fill_color=CLS_CMAP[latest_detection.class_id],
                    fill_opacity=0.8,
                )
                displayed_objects[stream_id][track_id] = marker
                marker_layer.add(marker)


Map(center=[39.975605748738964, -86.12710274800055], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In stream: trackletmerger:stream1, the total number of detections is 0
In stream: trackletmerger:stream2, the total number of detections is 0
In stream: trackletmerger:stream1, the total number of detections is 0
In stream: trackletmerger:stream2, the total number of detections is 0
In stream: trackletmerger:stream2, the total number of detections is 0
In stream: trackletmerger:stream1, the total number of detections is 0
In stream: trackletmerger:stream2, the total number of detections is 0
In stream: trackletmerger:stream1, the total number of detections is 0
In stream: trackletmerger:stream2, the total number of detections is 0
In stream: trackletmerger:stream1, the total number of detections is 0
In stream: trackletmerger:stream2, the total number of detections is 0
In stream: trackletmerger:stream1, the total number of detections is 0
In stream: trackletmerger:stream2, the total number of detections is 0
In stream: trackletmerger:stream1, the total number of detections is 0
In str